### Requirements
- Split the dataset into two tables
- Damage Stats: 1 containing: Name, Category, Phy, Mag, Fire, Ligh, Holy
- Level Requirements: 1 containing: Name, Str, Dex, Int, Fai, Arc
- For Damage Stats, if you look at the data in Phy, Mag, Fire, Ligh, Holy:
- -  the first value shows the attack damage 
- - the second value shows the damage resistance
- - A dash “-” means 0, i.e. no damage 
- For Level Requirements, for Str, Dex, Int, Fai, Arc:
- the first value shows the level required
- the second value shows the weapon scaling rating
- A dash “-” means 0, i.e. no requirement or scaling 
- Pivot the two datasets:
 - For Damage Stats,1 column containing Phy, Mag, Fire, Ligh, Holy and 1 for the values
 - For Level Requirements,1 column containing Str, Dex, Int, Fai, Arc and 1 for the values
 - Split the pivot values into 2 columns for both tables:
 - For Damage Stats label the first column “Attack Damage”, and the second “Damage Resistance”
 - For Level Requirements label the first column “Required Level”, and the second “Attribute Scaling” 
 - Replace the “-” values with zero in the columns: “Attack Damage”, “Damage Resistance”, “Required Level” and change the datatype to whole numbers (integer)
 - Find the total “Attack Damage” and total “Required Level” for all weapons and join the datasets together.
 - Rank the weapons by total attack damage, grouped by the total required attribute. 
 - Filter for the number 1 rank and output the data


In [1]:
import os
import pandas as pd

In [2]:
# Download sheet
# url = ' https://drive.google.com/file/d/1hWHg2TOW9eUZVlcmGopg1Biz7wjN9UF0/view'
# path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
# pd.read_csv(path).to_csv('preppin data week 37.csv')

df = pd.read_csv('preppin data week 37.csv').drop(columns='Unnamed: 0')

In [3]:
df.columns = [i.strip().lower().replace(' ','_') for i in df.columns]

# df.head()

 - Split the dataset into two tables
 - Damage Stats: 1 containing: Name, Category, Phy, Mag, Fire, Ligh, Holy
 - Level Requirements: 1 containing: Name, Str, Dex, Int, Fai, Arc

In [4]:
# Create 2 dfs
damage_df = df.loc[:,['name','category', 'phy', 'mag', 'fire', 'ligh', 'holy']]
level_requirement_df = df.loc[:,['name','str', 'dex', 'int', 'fai', 'arc']]

In [5]:
# Pivot tables

damage_df = pd.melt(damage_df, id_vars=['name','category'],value_vars=['phy', 'mag', 'fire', 'ligh', 'holy'])
damage_df[['attack_damage','damage_resistance']] =damage_df.value.str.split(' ',expand=True)
damage_df.drop(columns='value',inplace=True)
damage_df.replace('-',0,inplace=True)
damage_df[['attack_damage','damage_resistance']]=damage_df[['attack_damage','damage_resistance']].astype('int64')

damage_df= damage_df.groupby(['name','category']).sum('attack_damage')
damage_df.reset_index(inplace=True)


In [6]:

level_requirement_df=level_requirement_df.melt(id_vars='name',value_vars=['str', 'dex', 'int', 'fai', 'arc'])
level_requirement_df[['level_required','weapon_scaling_rating']] = level_requirement_df.value.str.split(expand=True)
level_requirement_df.drop(columns='value',inplace=True)
level_requirement_df.replace('-',0,inplace=True)
level_requirement_df['level_required']=level_requirement_df['level_required'].astype('int64')


level_requirement_df= level_requirement_df.groupby('name').sum('level_required').reset_index()

In [8]:
finaldf = damage_df.merge(level_requirement_df,on='name')

In [17]:
finaldf.set_index(['category']).sort_index().sort_values(['category','attack_damage'])

,name,attack_damage,damage_resistance,level_required
category,,,,
Axes,Ripple Blade,70,171,42
Axes,Forked Hatchet,101,148,23
Axes,Icerind Hatchet,111,148,27
Axes,Hand Axe,113,154,17
Axes,Warped Axe,118,204,32
...,...,...,...,...
Whips,Urumi,101,91,29
Whips,Thorned Whip,102,86,24
Whips,Hoslow's Petal Whip,110,96,30
